# Linear Programming Case Study: Happy Pet Food

### Problem statement

The Happy Pet Food Company manufactures two types of dog food: Meaties and Yummies. Each batch of Meaties contains 20 pounds of cereal and 20 pounds of meat; each batch of Yummies contains 30 pounds of cereal and 10 pounds of meat. Happy can buy only up to 400,000 pounds of cereal and up to 200,000 pounds of meat per month. Happy can only make a maximum of 8000 batches of Meaties each month due to sales constraints. Happy makes a profit of `$`65 on Meaties and `$`45 on Yummies. How many batches of Meaties and Yummies should it produce to maximize profit?

|          | Meaties X  | Yummies Y   | Resource constrains | 
| --------:| ---------- | ----------- | ------------------- | 
| Cereal   | 20         | 30          | 400,000             |
| Meat     | 20         | 10          | 200,000             | 
| Sales    | 1          |             | 8,000               |
| Profits  | 65         | 45          |                     |

The **decision variables** are the number of batches of Meaties as X. The number of batches of Yummies is Y.

### Objective function (maximize)

$$Z = 65x1 + 45x2$$

### Subject to contraints:

$$20x + 30y \leqslant 400,000$$
$$20x + 10y \leqslant 200,000$$
$$x \leqslant 8,000$$
$$x \geqslant 0$$
$$y \geqslant 0$$

### Graphical solution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

In [ ]:
x_i = np.linspace(0, 1000000, 1000000)
# 20x + 30y <= 400000
y_1 = (400000 - 20 * x_i) / 30
# 20x + 10y <= 300000
y_2 = (200000 - 20 * x_i) / 10
y_3 = np.minimum(y_1, y_2)

In [ ]:
plt.figure(figsize=(15, 8))

plt.plot(x_i, y_1, color = 'red', label=r'$20x + 30y\leq400000$')
plt.plot(x_i, y_2, color = 'green', label=r'$20x + 10y\leq200000$')
plt.axvline(x = 8000, color = 'black', label=r'$x\leq8000$')

plt.grid(True)
plt.xlim((0, 25000))
plt.ylim((0, 40000))
plt.xlabel('x')
plt.ylabel('y')

plt.legend(bbox_to_anchor=(1.15, 1), loc=2, borderaxespad=0.)
plt.fill_between(x_i[:8000], y_3[:8000], color = 'blue', alpha = 0.30)
plt.show()

### Pick corner point of feasible area

In [ ]:
opt_points = [(8000, 0), (8000, 4000), (5000, 10000), (0, 13333)]

for x,y in opt_points:
    print(f'At:{x, y}, Max(z)= {65*x + 45*y}')

### Using SciPy

In SciPy [linprog](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#linear-programming-example) only accepts a minimization problem if you're solving a maximizing problem then you've to re-write your equation so that maximize become minimize and greater than equal constraints become less than equal to constraint. It can done by multiplying the entire equation by -1.

### Create matrices to represent the objective function and the constraints

In [ ]:
# -z = -65x - 45y
objective = [-65, -45]

lhs_inequality = [[20, 30],
                  [20, 10]]

rhs_inequality = [400000,
                  200000]

bounds = [(0, 8000.0),
          (0, float('inf'))]

### Using Simplex method `method='simplex',`

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [ ]:
optimal_solution = linprog(c = objective, 
                           A_ub = lhs_inequality, 
                           b_ub = rhs_inequality,
                           bounds = bounds,
                           method = 'simplex')

optimal_solution

In [ ]:
optimal_solution.x

In [ ]:
optimal_solution.message

In [ ]:
optimal_solution.fun

## Using LpProblem function from PuLP library

This paper introduces the PuLP library, an open source package that allows mathematical programs to be described in the Python computer programming language. PuLP is a high-level modelling library that leverages the power of the Python language and allows the user to create programs using expressions that are natural to the Python language, avoiding special syntax and keywords wherever possible.

In [ ]:
!pip install pulp

In [ ]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

In [ ]:
model = LpProblem(name = 'happy-pet-food', sense = LpMaximize)

model

### Set variable and bound

In [ ]:
x = LpVariable(name='x', lowBound=0, upBound=8000)
y = LpVariable(name='y', lowBound=0)

### Set Constraints

In [ ]:
model += (20*x + 30*y <= 400000, 'Cereal constraints')
model += (20*x + 10*y <= 200000, 'Meat constraints')

model

### Set Objective function

In [ ]:
model += (65*x + 45*y)

model

### Status of solution

https://www.coin-or.org/PuLP/constants.html#pulp.constants.LpStatus

- LpStatusOptimal	“Optimal”	1
- LpStatusNotSolved	“Not Solved”	0
- LpStatusInfeasible	“Infeasible”	-1
- LpStatusUnbounded	“Unbounded”	-2
- LpStatusUndefined	“Undefined”	-3

In [ ]:
status = model.solve()

status

### Optimal value

In [ ]:
model.objective.value()

### Coordinates of optimal value

In [ ]:
for var in model.variables():
    print(f'{var.name} : {var.value()}')